In [1]:
from pathlib import Path

import igraph as ig
import leidenalg
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 5)
pd.set_option("display.max_colwidth", 10)
pd.set_option("display.width", 100)

# data

In [ ]:
local_data_dir = "data"
Path(local_data_dir).mkdir(parents=True, exist_ok=True)
raw_file_path   = Path(local_data_dir) / 'raw.snappy.parquet'
trans_file_path = Path(local_data_dir) / 'transposed.snappy.parquet'
cor_file_path   = Path(local_data_dir) / 'cor.snappy.parquet'

Download the raw expression data from TCGA.

In [3]:
if not raw_file_path.exists():
    url = 'https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.LUAD.sampleMap%2FHiSeqV2.gz'
    df = pd.read_csv(url, sep='\t', index_col=0) # ~10 secs
    df.to_parquet(raw_file_path, compression="snappy") # <1 sec
# df = pd.read_parquet(raw_file_path) # <1 sec
# print(df.shape)
# print(df.head())


Transpose the raw data so that rows are samples and columns are genes 

In [4]:
if not trans_file_path.exists():
    df = pd.read_parquet(raw_file_path) # <1 sec
    df = df.T  
    df.index.name = None
    df.columns.name = None
    df.to_parquet(trans_file_path, compression="snappy") # ~5 secs
df = pd.read_parquet(trans_file_path) # ~2 secs
print(df.shape)
print(df.head())


(576, 20530)
            ARHGEF10L    HIF3A  ...    SELP     SELS
TCGA-69...     9.9898   4.2598  ...  6.1209   9.8977
TCGA-62...    10.4257  11.6239  ...  8.6398   9.7315
TCGA-78...     9.6264   9.1362  ...  6.0970  10.3540
TCGA-50...     8.6835   9.4824  ...  9.5115  10.4914
TCGA-73...     9.2078   5.0288  ...  8.4187  10.3142

[5 rows x 20530 columns]


Check column dtypes

In [5]:
df.dtypes.value_counts()

float64    20530
Name: count, dtype: int64

Remove any genes with missing values.

In [6]:
p_missing = df.isna().mean() # count n missing values per gene
max_p_missing = 0
n_before = df.shape[1]
df = df.loc[:, p_missing <= max_p_missing]
n_after = df.shape[1]
delta = n_before - n_after
print(f'n columns with p(missing) > {max_p_missing}: {delta} ({delta / n_before:.1%})') 

n columns with p(missing) > 0: 0 (0.0%)


Filter by max proprtion of zeros

In [7]:
p_zero = (df == 0).mean()
max_p_zero = 0.1
n_before = df.shape[1]
df = df.loc[:, p_zero <= max_p_zero]
n_after = df.shape[1]
delta = n_before - n_after
print(f'n columns with p(zero) > {max_p_zero}: {delta} ({delta / n_before:.1%})') 


n columns with p(zero) > 0.1: 4437 (21.6%)


Filter by max proprtion of modal value

In [8]:
p_mode = df.apply(lambda x: (x == x.mode()[0]).mean())
max_p_mode = 0.9
n_before = df.shape[1]
df = df.loc[:, p_mode <= max_p_mode]
n_after = df.shape[1]
delta = n_before - n_after
print(f'n columns with p(mode) > {max_p_mode}: {delta} ({delta / n_before:.1%})') 


n columns with p(mode) > 0.9: 0 (0.0%)


Filter by variance

In [9]:
variances = df.var()
min_variance = 1e-5
n_before = df.shape[1]
df = df.loc[:, variances >= min_variance]
n_after = df.shape[1]
delta = n_before - n_after
print(f'n columns with variance < {min_variance}: {delta} ({delta / n_before:.1%})') 


n columns with variance < 1e-05: 0 (0.0%)


Final shape

In [10]:
print(df.shape)

(576, 16093)


# correlation matrix

In [11]:
if not cor_file_path.exists():
    cor = np.corrcoef(df.to_numpy(), rowvar=False) # ~10 seconds
    cor = pd.DataFrame(cor, columns=df.columns, index=df.columns) # 0 seconds
    cor.to_parquet(cor_file_path, compression="snappy") # ~1 min
cor = pd.read_parquet(cor_file_path) # ~30 secs
print(f'cor memory usage: {cor.memory_usage(deep=True).sum() / (1024**3):.2f} GB')
print(cor.shape)
print(cor.head())

cor memory usage: 1.93 GB
(16093, 16093)
           ARHGEF10L     HIF3A  ...      SELP      SELS
ARHGEF10L   1.000000 -0.001065  ...  0.019883 -0.297757
HIF3A      -0.001065  1.000000  ...  0.307253 -0.058198
RNF10       0.023010 -0.006597  ... -0.068626  0.040607
RNF11      -0.118511  0.061659  ...  0.236403  0.011459
RNF13      -0.090997  0.081001  ...  0.354377  0.288642

[5 rows x 16093 columns]


In [12]:
assert cor.shape[0] == df.shape[1]

# threshold

In [13]:
# get upper triangle indices (excluding diagonal)
upper_tri_index = np.triu_indices_from(cor, k=1)

# get abs(cor) values for the upper triangle
upper_tri_abs_values = np.abs(cor.to_numpy()[upper_tri_index])
print(upper_tri_abs_values.min())
print(upper_tri_abs_values.max())

# compute the size of the upper triangle
N = len(upper_tri_abs_values)
assert N == ((cor.size - cor.shape[0]) / 2)

# compare thresholds
thresholds = np.arange(0, 1, 0.1)
for threshold in thresholds:
    n = (upper_tri_abs_values > threshold).sum()
    print(f'threshold: {threshold:.1f}, n: {int(n):,} ({n / N:.2%})')


2.232842773050576e-09
0.9856822587830166
threshold: 0.0, n: 129,484,278 (100.00%)
threshold: 0.1, n: 72,644,107 (56.10%)
threshold: 0.2, n: 33,393,903 (25.79%)
threshold: 0.3, n: 12,994,241 (10.04%)
threshold: 0.4, n: 4,354,570 (3.36%)
threshold: 0.5, n: 1,262,516 (0.98%)
threshold: 0.6, n: 326,015 (0.25%)
threshold: 0.7, n: 83,116 (0.06%)
threshold: 0.8, n: 19,126 (0.01%)
threshold: 0.9, n: 1,808 (0.00%)


# graph

In [14]:
threshold = 0.3
mask = upper_tri_abs_values > threshold
sources = upper_tri_index[0][mask] # row index
targets = upper_tri_index[1][mask] # col index
weights = upper_tri_abs_values[mask] # |cor| values

g = ig.Graph(
    n=cor.shape[0],
    edges=list(zip(sources, targets)),
    directed=False
)

g.es["weight"] = weights
g.vs["name"] = cor.index.tolist()


In [15]:
weights.min()

np.float64(0.3000000064447378)

In [16]:
weights.max()

np.float64(0.9856822587830166)

In [17]:
# check graph size
print(f'''
n nodes: {g.vcount():,}
n edges: {g.ecount():,}
''')


n nodes: 16,093
n edges: 12,994,241



# community detection

In [19]:
# raise Exception('deliberate')

In [20]:
components = g.connected_components()

In [21]:
louvain = g.community_multilevel(weights="weight")

In [22]:
leiden0 = g.community_leiden(weights="weight")

In [23]:
leiden1 = leidenalg.find_partition(g, leidenalg.ModularityVertexPartition, weights="weight")

In [24]:
leiden2 = leidenalg.find_partition(g, leidenalg.CPMVertexPartition, weights="weight")


In [25]:
leiden3 = leidenalg.find_partition(g, leidenalg.ModularityVertexPartition, initial_membership=louvain.membership, weights="weight")

In [26]:
leiden4 = leidenalg.find_partition(g, leidenalg.CPMVertexPartition, initial_membership=louvain.membership, weights="weight")

In [31]:
results = pd.DataFrame({
    "node": g.vs["name"],
    "components": components.membership,
    "louvain": louvain.membership,
    "leiden0": leiden0.membership,
    "leiden1": leiden1.membership,
    "leiden2": leiden2.membership,
    "leiden3": leiden3.membership,
    "leiden4": leiden4.membership,
})

In [32]:
results.head()

,node,components,...,leiden3,leiden4
0,ARHGEF10L,0,...,1,1942
1,HIF3A,0,...,0,12170
2,RNF10,0,...,1,11637
3,RNF11,0,...,1,14815
4,RNF13,0,...,1,2717


In [33]:
results.nunique() # count number of partitions

node          16093
components       22
louvain          27
leiden0       16093
leiden1          27
leiden2       16093
leiden3          27
leiden4       16093
dtype: int64